# Product Description Generation
In this project, you can generate descriptions of various apparel items like shirts, jeans, kurtas using RNNs. The descriptions of 10400 products for each apparel item has been used fro jabong.com to train the neural network. The Neural Network built will generate a product description for a product from the given sets of apparels i.e. shirts, jeans, shoes, watches and kurtas.
## Get the Data
The data consists of 10400 descriptions of different kurtas obtained from jabong.com with distinguishing features.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/kurtas.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[:]

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 6314
Number of scenes: 23725
Average number of sentences in each scene: 0.0
Number of lines: 23725
Average number of words in each line: 28.915700737618547

The sentences 0 to 10:
﻿Anouk Blue Suit Set: Celebrate your personal style and roots with this elegant blue kurta set from Anouk. Define your figure in this kurta set with a skinny belt for a modern take on this traditional garment.



Varanga Grey Printed Kurta Pant Set: Be the cynosure of all eyes by wearing grey kurta pant set by Varanga, This will keep you comfortable throughout the party. Pair it with high heels and look effortlessly chic and fashionable. This set has a soothing colour and will lend you a fresh look this season.



Anouk Multicoloured Printed Kurta: This brilliant kurta from Anouk will help you maintain an elegant look all year long. Dress up for your next family event by pairing this multi-coloured piece with your best Kolhapuris and simple jewellery.



## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [3]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    text = list(set(text))   
    index = range(len(text))    
    int_to_vocab = dict(zip(index, text))   
    vocab_to_int = dict(zip(text, index))       
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    token = {}
    token['.']="||period||"
    token[',']="||comma||"
    token['"']="||quotation_mark||"
    token[';']="||semicolon||"
    token['!']="||exclamation_mark||"
    token['?']="||question_mark||"
    token['(']="||left_parantheses||"
    token[')']="||right_parantheses||"
    token['--'] = "||dash||"
    token['\n'] = "||return||"
    
    return token

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [8]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    INPUT = tf.placeholder(tf.int32, shape=(None,None), name="input")
    TARGETS = tf.placeholder(tf.int32, shape=(None,None), name='targets')
    LEARNING_RATE = tf.placeholder(tf.float32, shape=None, name='learning_rate')
    return INPUT, TARGETS, LEARNING_RATE


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [9]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    lstm1 = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    lstm2 = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    lstm3 = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    lstm4 = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    cell = tf.contrib.rnn.MultiRNNCell([lstm1,lstm2,lstm3,lstm4])
    initial_state = cell.zero_state(batch_size, tf.float32)
    initial_state = tf.identity(initial_state, name='initial_state')
   
     
    return cell, initial_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [10]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensionstea
    :return: Embedded input.
    """
    # TODO: Implement Function
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [11]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    Outputs, States = tf.nn.dynamic_rnn(cell, inputs,dtype='float32')
    FinalState = tf.identity(States,name="final_state")
    return Outputs, FinalState


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [12]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    embed = get_embed(input_data, vocab_size, embed_dim)
    outputs, final_state = build_rnn(cell, embed)
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=None,
                                               weights_initializer=tf.truncated_normal_initializer(stddev=0.1),
                                               biases_initializer=tf.zeros_initializer())
    return logits, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For example, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [13]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    num_batches = int(len(int_text) / (batch_size * seq_length))
    in_data = np.array(int_text[: num_batches * batch_size * seq_length])
    out_data = np.array(int_text[1: num_batches * batch_size * seq_length + 1])
    out_data[-1] = in_data[0]

    in_batches = np.split(in_data.reshape(batch_size, -1), num_batches, 1)
    out_batches = np.split(out_data.reshape(batch_size, -1), num_batches, 1)
    return np.array(list(zip(in_batches, out_batches)))


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [15]:
# Number of Epochs
num_epochs = 3000
# Batch Size
batch_size = 512
# RNN Size
rnn_size = 256
# Embedding Dimension Size
embed_dim = 256
# Sequence Length
seq_length = 16
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 32

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './savekurtas'

### Build the Graph
Build the graph using the neural network you implemented.

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')
    

Epoch   0 Batch    0/96   train_loss = 8.292
Epoch   0 Batch   32/96   train_loss = 5.514
Epoch   0 Batch   64/96   train_loss = 5.462
Epoch   1 Batch    0/96   train_loss = 5.211
Epoch   1 Batch   32/96   train_loss = 4.938
Epoch   1 Batch   64/96   train_loss = 4.491
Epoch   2 Batch    0/96   train_loss = 3.999
Epoch   2 Batch   32/96   train_loss = 3.723
Epoch   2 Batch   64/96   train_loss = 3.248
Epoch   3 Batch    0/96   train_loss = 2.665
Epoch   3 Batch   32/96   train_loss = 2.504
Epoch   3 Batch   64/96   train_loss = 2.272
Epoch   4 Batch    0/96   train_loss = 1.971
Epoch   4 Batch   32/96   train_loss = 2.023
Epoch   4 Batch   64/96   train_loss = 1.933
Epoch   5 Batch    0/96   train_loss = 1.719
Epoch   5 Batch   32/96   train_loss = 1.809
Epoch   5 Batch   64/96   train_loss = 1.746
Epoch   6 Batch    0/96   train_loss = 1.598
Epoch   6 Batch   32/96   train_loss = 1.656
Epoch   6 Batch   64/96   train_loss = 1.626
Epoch   7 Batch    0/96   train_loss = 1.498
Epoch   7 

Epoch  61 Batch    0/96   train_loss = 0.992
Epoch  61 Batch   32/96   train_loss = 1.053
Epoch  61 Batch   64/96   train_loss = 1.023
Epoch  62 Batch    0/96   train_loss = 0.981
Epoch  62 Batch   32/96   train_loss = 1.059
Epoch  62 Batch   64/96   train_loss = 1.009
Epoch  63 Batch    0/96   train_loss = 0.984
Epoch  63 Batch   32/96   train_loss = 1.050
Epoch  63 Batch   64/96   train_loss = 1.013
Epoch  64 Batch    0/96   train_loss = 0.979
Epoch  64 Batch   32/96   train_loss = 1.064
Epoch  64 Batch   64/96   train_loss = 1.015
Epoch  65 Batch    0/96   train_loss = 0.990
Epoch  65 Batch   32/96   train_loss = 1.059
Epoch  65 Batch   64/96   train_loss = 1.018
Epoch  66 Batch    0/96   train_loss = 0.985
Epoch  66 Batch   32/96   train_loss = 1.045
Epoch  66 Batch   64/96   train_loss = 1.025
Epoch  67 Batch    0/96   train_loss = 0.988
Epoch  67 Batch   32/96   train_loss = 1.056
Epoch  67 Batch   64/96   train_loss = 1.041
Epoch  68 Batch    0/96   train_loss = 0.994
Epoch  68 

Epoch 122 Batch    0/96   train_loss = 0.958
Epoch 122 Batch   32/96   train_loss = 1.029
Epoch 122 Batch   64/96   train_loss = 0.985
Epoch 123 Batch    0/96   train_loss = 0.957
Epoch 123 Batch   32/96   train_loss = 1.022
Epoch 123 Batch   64/96   train_loss = 0.998
Epoch 124 Batch    0/96   train_loss = 0.953
Epoch 124 Batch   32/96   train_loss = 1.004
Epoch 124 Batch   64/96   train_loss = 0.991
Epoch 125 Batch    0/96   train_loss = 0.949
Epoch 125 Batch   32/96   train_loss = 1.014
Epoch 125 Batch   64/96   train_loss = 0.982
Epoch 126 Batch    0/96   train_loss = 0.942
Epoch 126 Batch   32/96   train_loss = 1.011
Epoch 126 Batch   64/96   train_loss = 0.989
Epoch 127 Batch    0/96   train_loss = 0.956
Epoch 127 Batch   32/96   train_loss = 1.009
Epoch 127 Batch   64/96   train_loss = 0.979
Epoch 128 Batch    0/96   train_loss = 0.945
Epoch 128 Batch   32/96   train_loss = 1.013
Epoch 128 Batch   64/96   train_loss = 0.976
Epoch 129 Batch    0/96   train_loss = 0.950
Epoch 129 

Epoch 183 Batch    0/96   train_loss = 0.943
Epoch 183 Batch   32/96   train_loss = 1.003
Epoch 183 Batch   64/96   train_loss = 0.964
Epoch 184 Batch    0/96   train_loss = 0.930
Epoch 184 Batch   32/96   train_loss = 0.997
Epoch 184 Batch   64/96   train_loss = 0.960
Epoch 185 Batch    0/96   train_loss = 0.948
Epoch 185 Batch   32/96   train_loss = 1.004
Epoch 185 Batch   64/96   train_loss = 0.960
Epoch 186 Batch    0/96   train_loss = 0.937
Epoch 186 Batch   32/96   train_loss = 1.000
Epoch 186 Batch   64/96   train_loss = 0.973
Epoch 187 Batch    0/96   train_loss = 0.932
Epoch 187 Batch   32/96   train_loss = 1.007
Epoch 187 Batch   64/96   train_loss = 0.979
Epoch 188 Batch    0/96   train_loss = 0.937
Epoch 188 Batch   32/96   train_loss = 1.008
Epoch 188 Batch   64/96   train_loss = 0.975
Epoch 189 Batch    0/96   train_loss = 0.938
Epoch 189 Batch   32/96   train_loss = 1.006
Epoch 189 Batch   64/96   train_loss = 0.967
Epoch 190 Batch    0/96   train_loss = 0.929
Epoch 190 

Epoch 244 Batch    0/96   train_loss = 0.915
Epoch 244 Batch   32/96   train_loss = 0.997
Epoch 244 Batch   64/96   train_loss = 0.965
Epoch 245 Batch    0/96   train_loss = 0.913
Epoch 245 Batch   32/96   train_loss = 0.990
Epoch 245 Batch   64/96   train_loss = 0.958
Epoch 246 Batch    0/96   train_loss = 0.920
Epoch 246 Batch   32/96   train_loss = 0.994
Epoch 246 Batch   64/96   train_loss = 0.957
Epoch 247 Batch    0/96   train_loss = 0.928
Epoch 247 Batch   32/96   train_loss = 0.981
Epoch 247 Batch   64/96   train_loss = 0.957
Epoch 248 Batch    0/96   train_loss = 0.923
Epoch 248 Batch   32/96   train_loss = 0.989
Epoch 248 Batch   64/96   train_loss = 0.966
Epoch 249 Batch    0/96   train_loss = 0.925
Epoch 249 Batch   32/96   train_loss = 0.988
Epoch 249 Batch   64/96   train_loss = 0.962
Epoch 250 Batch    0/96   train_loss = 0.919
Epoch 250 Batch   32/96   train_loss = 0.984
Epoch 250 Batch   64/96   train_loss = 0.965
Epoch 251 Batch    0/96   train_loss = 0.918
Epoch 251 

...

Epoch 2706 Batch   64/96   train_loss = 0.950
Epoch 2707 Batch    0/96   train_loss = 0.906
Epoch 2707 Batch   32/96   train_loss = 0.967
Epoch 2707 Batch   64/96   train_loss = 0.939
Epoch 2708 Batch    0/96   train_loss = 0.897
Epoch 2708 Batch   32/96   train_loss = 0.987
Epoch 2708 Batch   64/96   train_loss = 0.971
Epoch 2709 Batch    0/96   train_loss = 0.911
Epoch 2709 Batch   32/96   train_loss = 0.977
Epoch 2709 Batch   64/96   train_loss = 0.952
Epoch 2710 Batch    0/96   train_loss = 0.902
Epoch 2710 Batch   32/96   train_loss = 0.979
Epoch 2710 Batch   64/96   train_loss = 0.947
Epoch 2711 Batch    0/96   train_loss = 0.901
Epoch 2711 Batch   32/96   train_loss = 0.975
Epoch 2711 Batch   64/96   train_loss = 0.954
Epoch 2712 Batch    0/96   train_loss = 0.915
Epoch 2712 Batch   32/96   train_loss = 0.963
Epoch 2712 Batch   64/96   train_loss = 0.946
Epoch 2713 Batch    0/96   train_loss = 0.893
Epoch 2713 Batch   32/96   train_loss = 0.968
Epoch 2713 Batch   64/96   train_l

Epoch 2766 Batch   32/96   train_loss = 0.971
Epoch 2766 Batch   64/96   train_loss = 0.958
Epoch 2767 Batch    0/96   train_loss = 0.912
Epoch 2767 Batch   32/96   train_loss = 0.977
Epoch 2767 Batch   64/96   train_loss = 0.979
Epoch 2768 Batch    0/96   train_loss = 0.920
Epoch 2768 Batch   32/96   train_loss = 0.972
Epoch 2768 Batch   64/96   train_loss = 0.967
Epoch 2769 Batch    0/96   train_loss = 0.917
Epoch 2769 Batch   32/96   train_loss = 0.971
Epoch 2769 Batch   64/96   train_loss = 0.957
Epoch 2770 Batch    0/96   train_loss = 0.915
Epoch 2770 Batch   32/96   train_loss = 0.966
Epoch 2770 Batch   64/96   train_loss = 0.988
Epoch 2771 Batch    0/96   train_loss = 0.914
Epoch 2771 Batch   32/96   train_loss = 0.971
Epoch 2771 Batch   64/96   train_loss = 0.985
Epoch 2772 Batch    0/96   train_loss = 0.928
Epoch 2772 Batch   32/96   train_loss = 0.990
Epoch 2772 Batch   64/96   train_loss = 0.964
Epoch 2773 Batch    0/96   train_loss = 0.926
Epoch 2773 Batch   32/96   train_l

Epoch 2826 Batch    0/96   train_loss = 0.904
Epoch 2826 Batch   32/96   train_loss = 0.972
Epoch 2826 Batch   64/96   train_loss = 0.946
Epoch 2827 Batch    0/96   train_loss = 0.922
Epoch 2827 Batch   32/96   train_loss = 0.976
Epoch 2827 Batch   64/96   train_loss = 0.965
Epoch 2828 Batch    0/96   train_loss = 0.924
Epoch 2828 Batch   32/96   train_loss = 0.990
Epoch 2828 Batch   64/96   train_loss = 0.961
Epoch 2829 Batch    0/96   train_loss = 0.896
Epoch 2829 Batch   32/96   train_loss = 0.977
Epoch 2829 Batch   64/96   train_loss = 0.955
Epoch 2830 Batch    0/96   train_loss = 0.898
Epoch 2830 Batch   32/96   train_loss = 0.979
Epoch 2830 Batch   64/96   train_loss = 0.957
Epoch 2831 Batch    0/96   train_loss = 0.902
Epoch 2831 Batch   32/96   train_loss = 0.974
Epoch 2831 Batch   64/96   train_loss = 0.980
Epoch 2832 Batch    0/96   train_loss = 0.917
Epoch 2832 Batch   32/96   train_loss = 0.999
Epoch 2832 Batch   64/96   train_loss = 0.973
Epoch 2833 Batch    0/96   train_l

Epoch 2885 Batch   64/96   train_loss = 0.944
Epoch 2886 Batch    0/96   train_loss = 0.923
Epoch 2886 Batch   32/96   train_loss = 0.974
Epoch 2886 Batch   64/96   train_loss = 0.967
Epoch 2887 Batch    0/96   train_loss = 0.933
Epoch 2887 Batch   32/96   train_loss = 0.990
Epoch 2887 Batch   64/96   train_loss = 0.949
Epoch 2888 Batch    0/96   train_loss = 0.926
Epoch 2888 Batch   32/96   train_loss = 0.989
Epoch 2888 Batch   64/96   train_loss = 0.941
Epoch 2889 Batch    0/96   train_loss = 0.906
Epoch 2889 Batch   32/96   train_loss = 0.980
Epoch 2889 Batch   64/96   train_loss = 0.941
Epoch 2890 Batch    0/96   train_loss = 0.902
Epoch 2890 Batch   32/96   train_loss = 1.000
Epoch 2890 Batch   64/96   train_loss = 0.947
Epoch 2891 Batch    0/96   train_loss = 0.914
Epoch 2891 Batch   32/96   train_loss = 1.004
Epoch 2891 Batch   64/96   train_loss = 0.958
Epoch 2892 Batch    0/96   train_loss = 0.926
Epoch 2892 Batch   32/96   train_loss = 0.987
Epoch 2892 Batch   64/96   train_l

Epoch 2945 Batch   32/96   train_loss = 0.975
Epoch 2945 Batch   64/96   train_loss = 0.935
Epoch 2946 Batch    0/96   train_loss = 0.912
Epoch 2946 Batch   32/96   train_loss = 0.970
Epoch 2946 Batch   64/96   train_loss = 0.960
Epoch 2947 Batch    0/96   train_loss = 0.915
Epoch 2947 Batch   32/96   train_loss = 0.987
Epoch 2947 Batch   64/96   train_loss = 0.951
Epoch 2948 Batch    0/96   train_loss = 0.930
Epoch 2948 Batch   32/96   train_loss = 0.979
Epoch 2948 Batch   64/96   train_loss = 0.936
Epoch 2949 Batch    0/96   train_loss = 0.912
Epoch 2949 Batch   32/96   train_loss = 0.982
Epoch 2949 Batch   64/96   train_loss = 0.947
Epoch 2950 Batch    0/96   train_loss = 0.909
Epoch 2950 Batch   32/96   train_loss = 0.967
Epoch 2950 Batch   64/96   train_loss = 0.952
Epoch 2951 Batch    0/96   train_loss = 0.905
Epoch 2951 Batch   32/96   train_loss = 0.962
Epoch 2951 Batch   64/96   train_loss = 0.972
Epoch 2952 Batch    0/96   train_loss = 0.912
Epoch 2952 Batch   32/96   train_l

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [3]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    input_tensor = loaded_graph.get_tensor_by_name('input:0')
    initial_state_tensor = loaded_graph.get_tensor_by_name('initial_state:0')
    final_state_tensor = loaded_graph.get_tensor_by_name('final_state:0')
    probs_tensor = loaded_graph.get_tensor_by_name('probs:0')
    
    return input_tensor, initial_state_tensor, final_state_tensor, probs_tensor



tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [4]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    next_word = np.random.choice(len(int_to_vocab), p=probabilities)
    prediction = int_to_vocab[next_word]
    return prediction




tests.test_pick_word(pick_word)

Tests Passed


## Generate Product Description
This will generate the Description for your product.  Set `gen_length` to the length of product description you want to generate.

In [7]:
import webcolors
import helper
import time

def remove_duplicates(li):
    my_set = set()
    res = []
    for e in li:
        if e not in my_set:
            res.append(e)
            my_set.add(e)

    return res

def get_brandname():

    data_dir = './data/simpsons/kurtas.txt'
    text = helper.load_data(data_dir)
    # Ignore notice, since we don't use it for analysing the data
    text = text[:]
    brands = []
    alldesc = text.split("\n\n")
    for eachdesc in alldesc:
        eachtitle = eachdesc.split(":")[0]
        brandname = eachtitle.split(" ")[0]
        brandname = brandname.replace("\n",'')
        brandname = brandname.replace("\ufeff",'')
        brandname = brandname.lower()
        brands.append(brandname)
    brands = set(brands)
    brands = list(brands)

    brands.remove('and')
    brands.remove('')
    brands.remove('no')
    brands.remove('only')
    brands.remove('colour')
    brands.remove('all')
    brands.append('about')
    brands.append('you')
    clrlst = get_colors()
    brands = list(set(brands)-set(clrlst))

    return brands

def get_colors():
    
    list_of_colors = []
    listcolors = webcolors.CSS3_NAMES_TO_HEX
    for all in listcolors:
        cl = all.split(":")[0]
        list_of_colors.append(cl)
    list_of_colors.append('mauve')
    list_of_colors.append('wine')
    return list_of_colors

def get_style():
    list_of_styles = ['skinny', 'slim', 'regular','washed','comfort']
    return list_of_styles

def changetemplate(desc_op,features_str):
    
    #Getting brand names from text files to identify brand in the features string
    brandnames = get_brandname()
    brand_instr = ''
    for allbrands in brandnames:
        for allwords in features_str.split(" "):
            if allbrands == allwords:
                brand_instr = allbrands
                break
                

    #getting all colors to identify the color in the features string
    colorlist = get_colors()
    color_instr = ''
    for allcolors in colorlist:
        for allword in desc_op.split(":")[0].split(" "):
            if allcolors == allword:
                color_instr = allcolors
                break
    for allcolors in colorlist:
        for allwords in features_str.split(" "):
            if allcolors == allwords:
                color_instr = allcolors
                break
                
    #getting all styles to identify the styles in the features string
    stylelist = get_style()
    style_instr = ''
    for allcolors in stylelist:
        for allword in desc_op.split(":")[0].split(" "):
            if allcolors == allword:
                style_instr = allcolors
                break
    for allcolors in stylelist:
        for allwords in features_str.split(" "):
            if allcolors == allwords:
                style_instr = allcolors
                break




    # to append the brand, color and style in the title if not present

    allwordsintitle = desc_op.split(":")[0]
    
    for allwords in allwordsintitle.split(" "):
        if allwords == style_instr:
            flg = 0
        else:
            allwordsintitle = style_instr + " " + allwordsintitle
            title = desc_op.split(":")[0]
            allwordsintitle = title.replace(desc_op.split(":")[0],allwordsintitle)
            break

    for allwords in allwordsintitle.split(" "):
        if allwords == color_instr:
            flg = 0
        else:
            allwordsintitle = color_instr + " " + allwordsintitle
            title = desc_op.split(":")[0]
            allwordsintitle = title.replace(desc_op.split(":")[0],allwordsintitle)
            break

    for allwords in allwordsintitle.split(" "):
        if allwords == brand_instr:
            flg = 0
        else:
            allwordsintitle = brand_instr + " " + allwordsintitle
            title = desc_op.split(":")[0]
            allwordsintitle = title.replace(desc_op.split(":")[0],allwordsintitle)
            break
            
    


    #to replace brand names and colours in the description 

    allwordsindescription = desc_op.split(":")[1]

    for brandnm in brandnames:
        for allwords in allwordsintitle.split(" "):
            if brandnm == allwords or brandnm+"." == allwords or brandnm+"," == allwords or brandnm+";" == allwords:
                allwordsintitle = allwordsintitle.replace(brandnm,brand_instr)
        for allwords in allwordsindescription.split(" "):
            if brandnm == allwords or brandnm+"." == allwords or brandnm+"," == allwords or brandnm+";" == allwords:
                allwordsindescription = allwordsindescription.replace(brandnm,brand_instr)

    
    allwordsindescriptionlist = allwordsindescription.split(" ")
    allwordsindescriptionlist = remove_duplicates(allwordsindescriptionlist)
    allwordsindescription = ' '.join(allwordsindescriptionlist)
    
    desc_op = desc_op.replace(desc_op.split(":")[0],allwordsintitle)
    desc_op = desc_op.replace(desc_op.split(":")[1],allwordsindescription)


    for clrs in colorlist:
        for allwords in allwordsintitle.split(" "):
            if clrs == allwords or clrs+"." == allwords or clrs+"," == allwords or clrs+";" == allwords:
                allwordsintitle = allwordsintitle.replace(clrs,color_instr)
        for allwords in allwordsindescription.split(" "):
            if clrs == allwords or clrs+"." == allwords or clrs+"," == allwords or clrs+";" == allwords:
                allwordsindescription = allwordsindescription.replace(clrs,color_instr)


    allwordsintitlelist = allwordsintitle.split(" ")
    allwordsintitlelist = remove_duplicates(allwordsintitlelist)
    allwordsintitle = ' '.join(allwordsintitlelist)

    desc_op = desc_op.replace(desc_op.split(":")[0],allwordsintitle)
    desc_op = desc_op.replace(desc_op.split(":")[1],allwordsindescription)
    
    for brandnm in stylelist:
        for allwords in allwordsintitle.split(" "):
            if brandnm == allwords or brandnm+"." == allwords or brandnm+"," == allwords or brandnm+";" == allwords:
                allwordsintitle = allwordsintitle.replace(brandnm,style_instr)
        for allwords in allwordsindescription.split(" "):
            if brandnm == allwords or brandnm+"." == allwords or brandnm+"," == allwords or brandnm+";" == allwords:
                allwordsindescription = allwordsindescription.replace(brandnm,style_instr)

    
    allwordsindescriptionlist = allwordsindescription.split(" ")
    allwordsindescriptionlist = remove_duplicates(allwordsindescriptionlist)
    allwordsindescription = ' '.join(allwordsindescriptionlist)
    
    desc_op = desc_op.replace(desc_op.split(":")[0],allwordsintitle)
    desc_op = desc_op.replace(desc_op.split(":")[1],allwordsindescription)
    print(desc_op+"\n\n")


gen_length = 500

features_str = 'shree green kurta'

brandnamesall = get_brandname()
prime_word = ''
for allb in brandnamesall:
    for wrds in features_str.split(" "):
        if allb == wrds:
            prime_word = allb
            break


loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word]
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
    
descriptions = tv_script.split("\n\n")
desc_op = ''
for description in descriptions:
    try:
        x = description.split(":")[0]
        y = description.split(":")[1]
        desc = description
        descls = desc.split()
        desc = ' '.join(descls)
        #print(desc+"\n\n")
        changetemplate(desc,features_str)
        
    except:
        flag = 0
    

INFO:tensorflow:Restoring parameters from ./savekurtas
shree green  printed kurta: flaunt sartorial elegance when shree wear this off-white coloured kurta by shree. look classy and stylish in piece revel the of its soft cotton fabric. worn with contrast leggings, it will go well palazzos kolhapuris. mandarin collar full button placket allover contemporary print colourblocking three-quarter sleeves side slits


shree green  round neck a line kurta with 3/4th sleeves: exhibiting an open-front kurta with contrasting print all over, this green anarkali from shree will enhance your ethnic look effortlessly. 3/4th-sleeved shirt-style coloured jacket set be well or strappy heels to best. split neckline taping on the front bold floral motifs over straight cut


shree green  peach printed kurta: available in striking contrast geometric print all over, this green kurta from shree is sure to capture the fancy, making it epitome of elegance. adding wow appeal printed will go well with a pair leggi